<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [6]:
pip install polars

Note: you may need to restart the kernel to use updated packages.


In [7]:
import polars as pl
import random
import string

# Generacion de Datos

In [9]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""ybdwa""",20,55.47038,"2022-12-04 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2221
2,"""gveja""",10,99.215666,"2018-03-17 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3410
3,"""miguk""",10,22.87259,"2016-02-07 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4957
4,"""agxec""",17,75.029096,"2012-02-10 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4354
5,"""femep""",18,63.895379,"2012-11-09 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2473


In [10]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [11]:
lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación

resultado = (
    lazy_df
    .filter(pl.col("cantidad")<5)
    .with_columns([(pl.col("cantidad")*pl.col("precio_unitario")).alias("total_venta")])
    .drop("precio_unitario")
    .group_by("producto").agg([
        pl.col("total_venta").sum().alias("total_vendido"),
        pl.col("cantidad").sum().alias("cantidad_total_vendida")
    ])
    .sort("total_vendido")

)

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)

shape: (209, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ azszj    ┆ 12.066865     ┆ 1                      │
│ vnnrt    ┆ 19.992323     ┆ 1                      │
│ tfppp    ┆ 20.111257     ┆ 2                      │
│ nsrow    ┆ 22.171963     ┆ 1                      │
│ dkqvb    ┆ 22.844126     ┆ 2                      │
│ …        ┆ …             ┆ …                      │
│ vmvkf    ┆ 349.781292    ┆ 4                      │
│ idzbb    ┆ 360.36772     ┆ 4                      │
│ sknke    ┆ 373.405188    ┆ 4                      │
│ zisln    ┆ 380.521356    ┆ 4                      │
│ tozly    ┆ 380.834225    ┆ 4                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [12]:
# Tu codigo aqui
# Save the DataFrame to a Parquet file
resultado_df.write_parquet("resultados.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [13]:
# Tu codigo aqui
# Read the Parquet file into a Polars DataFrame
df = pl.read_parquet("resultados.parquet")

# Display the DataFrame
print(df)

shape: (209, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ azszj    ┆ 12.066865     ┆ 1                      │
│ vnnrt    ┆ 19.992323     ┆ 1                      │
│ tfppp    ┆ 20.111257     ┆ 2                      │
│ nsrow    ┆ 22.171963     ┆ 1                      │
│ dkqvb    ┆ 22.844126     ┆ 2                      │
│ …        ┆ …             ┆ …                      │
│ vmvkf    ┆ 349.781292    ┆ 4                      │
│ idzbb    ┆ 360.36772     ┆ 4                      │
│ sknke    ┆ 373.405188    ┆ 4                      │
│ zisln    ┆ 380.521356    ┆ 4                      │
│ tozly    ┆ 380.834225    ┆ 4                      │
└──────────┴───────────────┴────────────────────────┘
